In [8]:
# Suprimir advertencias
import warnings
warnings.filterwarnings("ignore")

#Importing Libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
import datetime
from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency

In [10]:
#Loading Data:
data = pd.read_csv("IFPTML_MCSD.csv", sep=';')

In [4]:
# CHECKING DATA
data["prediction_trainig"] = data["prediction_trainig"].str.strip()
print(len(data[data["prediction_trainig"] == "training"]))
print(len(data[data["prediction_trainig"] == "validation"]))

1308
393


In [5]:
#Data Splitting:
Y = data.iloc[:, 1]
X = data.iloc[:, 3:]

In [6]:
#Splitting Data into Training and Prediction Sets:
train_data = data[data["prediction_trainig"] == "training"]
pred_data = data[data["prediction_trainig"] == "validation"]

In [8]:
# Model Preparation for LDA
X_train_LDA= train_data.iloc[:, 3:]
X_test_LDA= pred_data.iloc[:, 3:]
y_train_LDA= train_data.iloc[:, 1]
y_test_LDA= pred_data.iloc[:, 1]

print(X_train_LDA.shape)
print(X_test_LDA.shape)
print(y_train_LDA.shape)
print(y_test_LDA.shape)

(1308, 10)
(393, 10)
(1308,)
(393,)


In [14]:
#X_LDA
mlr_X_train_LDA= X_train_LDA[['fref', 'sIcI', 'sIcIV','sIIcIV','sIIIcII', 'sIIIcIII', 'sIIIcvII','sIVcIV','sIVcIV','sIVcIV','sVIcvI']]
mlr_X_test_LDA= X_test_LDA[['fref', 'sIcI', 'sIcIV','sIIcIV','sIIIcII', 'sIIIcIII', 'sIIIcvII','sIVcIV','sIVcIV','sIVcIV','sVIcvI']]

In [16]:
#Model_LDA

In [17]:
# Define the Prior Classification Probabilities
priors = [0.5, 0.5]  

In [18]:
# Initialize the LDA model with the prior probabilities
LDA = LinearDiscriminantAnalysis(priors=priors)

In [22]:
#Fit the model

In [24]:
LDA.fit(mlr_X_train_LDA, y_train_LDA)

LinearDiscriminantAnalysis(priors=[0.5, 0.5])

In [26]:
y_pred_train_LDA = LDA.predict(mlr_X_train_LDA)
y_pred_test_LDA = LDA.predict(mlr_X_test_LDA)

In [28]:
#Perform cross-validation with LinearDiscriminantAnalysis
cv_scores = cross_val_score(LDA, mlr_X_train_LDA, y_train_LDA, cv=5)  # 5-fold cross-validation
print("Cross-validation Scores:", cv_scores)

Cross-validation Scores: [0.70992366 0.79389313 0.85114504 0.71264368 0.88505747]


In [30]:
# Get variable names
variable_names = X.columns

In [32]:
# Coefficients of the linear discriminant function
coefficients = LDA.coef_[0]
intercept = LDA.intercept_[0]

In [34]:
# Imprimir la ecuación del modelo de forma segura
print("Equation of the line:")
print("y = ", end="")

# zip() se detiene en el tamaño más corto entre las dos listas, evitando errores de índice
for coefficient, variable_name in zip(coefficients, variable_names):
    print(f"({coefficient:.3f} * {variable_name}) + ", end="")

# Imprimir la constante del modelo
print(f"{intercept:.3f}")

Equation of the line:
y = (11.811 * fref) + (-0.380 * sIcI) + (2.393 * sIcIV) + (-0.982 * sIIcIV) + (0.534 * sIIIcII) + (-0.002 * sIIIcIII) + (-0.284 * sIIIcvII) + (0.380 * sIVcIV) + (0.380 * sVcv) + (0.380 * sVIcvI) + -4.628


In [36]:
confusion_train = confusion_matrix(y_train_LDA,y_pred_train_LDA)
confusion_test = confusion_matrix(y_test_LDA, y_pred_test_LDA)

In [38]:
y_train_pred_proba = LDA.predict_proba(mlr_X_train_LDA)
y_test_pred_proba = LDA.predict_proba(mlr_X_test_LDA)

In [40]:
print('Confusion Matrix for Training Series')
print(confusion_train)
print('Confusion Matrix for Test Series')
print(confusion_test)

Confusion Matrix for Training Series
[[843 198]
 [ 57 210]]
Confusion Matrix for Test Series
[[252  84]
 [  9  48]]


In [42]:
tn, fp, fn, tp = confusion_matrix(y_train_LDA, y_pred_train_LDA).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tn+tp)/(tn+tp+fn+fp)

In [44]:
print('Sp(%)=', specificity,'Sn(%)=' , sensitivity,'Ac(%)=' , accuracy)

Sp(%)= 0.8097982708933718 Sn(%)= 0.7865168539325843 Ac(%)= 0.805045871559633


In [46]:
tn, fp, fn, tp = confusion_matrix(y_test_LDA, y_pred_test_LDA).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tn+tp)/(tn+tp+fn+fp)

In [48]:
print('Sp(%)=', specificity,'Sn(%)=' , sensitivity,'Ac(%)=' , accuracy),

Sp(%)= 0.75 Sn(%)= 0.8421052631578947 Ac(%)= 0.7633587786259542


(None,)